# Tutorial about rendering LocData

## Setting things up

In [ ]:
from pathlib import Path

%matplotlib inline
# %matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

### Simulated data

Localizations are simulated that are distributed according to a Neyman-Scott distribution (blobs).

In [ ]:
dat = lc.simulate_Thomas(n_samples=1000, n_features=2, centers=10, feature_range=(0, 1000), cluster_std=10, seed=0)
dat.print_summary()

## Show scatter plot

Since localization data is kept as a pandas dataframe standard plotting routines from pandas or matplotlip can be used.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
dat.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Render by simple 2D binning

A method for simply binning localization data in 2D pixels is provided. This function uses the fast_histogram library.

In [ ]:
img, bins, label = lc.histogram(dat, bin_size=10)

In [ ]:
plt.hist(img.ravel(), bins=256, range=(1, 50), fc='k', ec='k');

The `render_2d` method can directly provide a new figure as output.

In [ ]:
lc.render_2d(dat, bin_size=10);

Or it can be used within the matplotlib environment.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
lc.render_2d(dat, ax = ax, bin_size=10, cmap='viridis')
plt.show()

Intensity is per default scaled to the min and max intensity values but can be rescaled to a certain percentile between the min and max intensities:

In [ ]:
lc.render_2d(dat, bin_size=10, rescale=(0, 50));

The image size is set automatically to the min and max coordinates but can be set to (0, max) or an arbitrary range.

In [ ]:
lc.render_2d(dat, bin_size=10, bin_range='zero');

In [ ]:
lc.render_2d(dat, bin_size=10, bin_range=((0,400),(0,200)));

## Use different libraries for rendering

Rendering can also be carried out with a different render engine. Choose one of the following (MPL is the standard matplotlib):

In [ ]:
list(lc.RenderEngine)

### napari

As external viewer you can use **napari**.

For napari the qt gui interface has to be activated. Make sure to run the following command manually in a single cell and give enough time for this procedure to complete in the background.

or alternatively:

Remember: In any python script the command has to run within the `napari.gui_qt` context:

`
with napari.gui_qt():
    viewer = lc.render_2d(dat, render_engine=lc.RenderEngine.NAPARI)
`

### mpl-scatter-density

For interactive viewing with variable binning, **mpl-scatter-density** is helpful. To run this the notebook has to be run with a new kernel applying the magic command `%matplotlib widget`  before matplotib is imported.

In [ ]:
lc.render_2d(dat, render_engine=lc.RenderEngine.MPL_SCATTER_DENSITY);